In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import textwrap
import simple_llm_cache
import llm_cache_stats_wrapper
import os

# In order to make it easy to run work projects and personal AI experiments, override OPENAI_API_KEY with the value of OPENAI_API_KEY_PERSONAL if it is set.
if "OPENAI_API_KEY_PERSONAL" in os.environ:
    print("Using key from OPENAI_API_KEY_PERSONAL environment variable")
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY_PERSONAL"]
if "ANTHROPIC_API_KEY_PERSONAL" in os.environ:
    print("Using key from ANTHROPIC_API_KEY_PERSONAL environment variable")
    os.environ["ANTHROPIC_API_KEY"] = os.environ["ANTHROPIC_API_KEY_PERSONAL"]

verbose = False
temperature = 0.5

langchain.llm_cache = llm_cache_stats_wrapper.LlmCacheStatsWrapper(simple_llm_cache.SimpleLlmCache("llm-cache.json"))

def dump_cache_stats_since_last_call():
    print(langchain.llm_cache.get_cache_stats_summary())
    langchain.llm_cache.clear_cache_stats()

template = """Answer the following question as if you are a {character} character:
{question}
"""

prompt = prompt = PromptTemplate(
    input_variables=["character", "question"],
    template=template)

for model_name in ["gpt-3.5-turbo", "gpt-4", "gpt-4o", "claude-3-haiku-20240307"]:
    if model_name.startswith("gpt-"):
        llm = ChatOpenAI(
            temperature=temperature,
            model_name = model_name)
    elif model_name.startswith("claude-"):
        llm = ChatAnthropic(
            temperature=temperature,
            model_name = model_name)

    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose)

    for trial in range(2):
        print(f"\n*** {model_name} trial {trial} ***")
        langchain.llm_cache.inner_cache.set_trial(trial)
        output = chain.predict(
            character="pirate",
            question="What is a neural network?")
        print(textwrap.fill(output, width=80))

print()
dump_cache_stats_since_last_call()


*** claude-3-haiku-20240307 trial 0 ***


c:\Users\forre\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\forre\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


Arr, ye be askin' about the mysteries o' the neural network, eh? Well, me
hearty, let ol' Blackbeard explain it to ye in the language o' the high seas.  A
neural network be like a fleet o' ships, each one representin' a neuron in yer
brain. These ships be connected by ropes, an' the strength o' these ropes be the
weights o' the connections. When ye feed yer neural network some information, it
be like the ships be sendin' signals back an' forth, tryin' to figure out the
best way to navigate the treacherous waters o' yer problem.  The more ships ye
have in yer fleet, an' the stronger the ropes be connectin' 'em, the better yer
neural network can learn an' adapt to the challenges it faces. It be like a crew
o' seasoned pirates, workin' together to navigate the uncharted seas o' data an'
find the buried treasure o' the answer.  Arr, an' just like a pirate ship, the
neural network can be trained to do all sorts o' tasks, from recognizin' the
faces o' yer shipmates to predictin' the weather 